In [340]:
import os
import pandas as pd

In [341]:
#Load Train and Test Data
URL_TRAIN = "https://www.mxhackathon.co.za/docs/TrainData.csv"
URL_TEST  = "https://www.mxhackathon.co.za/docs/TestData.csv"
dfTrain = pd.read_csv(URL_TRAIN)
dfTest = pd.read_csv(URL_TEST)

In [342]:
mappings = {}

In [343]:
# Define function to clean and engineer the data
def clean_train_data(df):
    # Drop CustomerID
    df = df.drop("CustomerID", axis=1)

    #Drop LeadID
    df = df.drop("LeadID", axis=1)

    # Converting DTLeadCreated to type DateTime
    df["DTLeadCreated"] = pd.to_datetime(df["DTLeadCreated"])
    df["DayOfWeek"] = df["DTLeadCreated"].dt.dayofweek # Extracting The Day of the Week from the date
    
    # Converting DRLeadAllocated to type DateTime
    df["DTLeadAllocated"] = pd.to_datetime(df["DTLeadAllocated"])
    
    # Calculating and storing the difference between lead creation and lead allocation
    df["AllocationTime"] = (df["DTLeadAllocated"] - df["DTLeadCreated"]).dt.total_seconds()/60
    
    # Dropping date Lead Created and date lead allocated
    df = df.drop("DTLeadCreated", axis=1)
    df = df.drop("DTLeadAllocated", axis=1)

    # Method to extract the deaer locations from the dealer name
    def get_dealer_location(dealer_string):

        #remove the suffix and change to lowercase
        dealer_string = dealer_string.lower().split()

        if "multifranchise" in dealer_string:
            dealer_name = dealer_string[0]

            # check the common two word cities at the end
            if dealer_name == "cape" or dealer_name == "east" or dealer_name == "west":
                dealer_name = dealer_name + " " + dealer_string[1]

        elif "motus" in dealer_string:
            dealer_name = dealer_string[-1]

            # check the common two word cities at the end
            if dealer_name == "town" or dealer_name == "rand":
                dealer_name = dealer_string[-2] + " " + dealer_name

        else:
            dealer_name = dealer_string[-1]

            # check the common two word cities at the end
            if dealer_name == "town" or dealer_name == "rand":
                dealer_name = dealer_string[-2] + " " + dealer_name

        # check if a number was found
        if dealer_name.isdigit():
            return dealer_string[-3]
        else:
            return dealer_name


    # Extract the Dealer's city
    df["DealerCity"] =df["Dealer"].apply(get_dealer_location)

    # Convert String features into type Category and float for the scikit Model to work with
    dealer_categories = df["Dealer"].astype("category").cat.categories
    dealer_mapping = {cat: idx for idx, cat in enumerate(dealer_categories)}
    df["Dealer"] = df["Dealer"].map(dealer_mapping)
    
    dealer_city_categories = df["DealerCity"].astype("category").cat.categories
    dealer_city_mapping = {cat: idx for idx, cat in enumerate(dealer_city_categories)}
    df["DealerCity"] = df["DealerCity"].map(dealer_city_mapping)

    # Convert String features into type Category and float for the scikit Model to work with
    lead_type_categories = df["LeadType"].astype("category").cat.categories
    lead_type_mapping = {cat: idx for idx, cat in enumerate(lead_type_categories)}
    df["LeadType"] = df["LeadType"].map(lead_type_mapping)
    
    lead_source_categories = df["LeadSource"].astype("category").cat.categories
    lead_source_mapping = {cat: idx for idx, cat in enumerate(lead_source_categories)}
    df["LeadSource"] = df["LeadSource"].map(lead_source_mapping)

    # Convert String features into type Category and float for the scikit Model to work with
    seek_categories = df["Seek"].astype("category").cat.categories
    seek_mapping = {cat: idx for idx, cat in enumerate(seek_categories)}
    df["Seek"] = df["Seek"].map(seek_mapping)

    # Method to get cleaned InterestMake
    def clean_interest_make(interest_make):
      interest_make = str(interest_make).upper().split(" ")
      interest_make = interest_make[0].split("-")
      return interest_make[0]

    # Clean InterestMake feature
    df["InterestMake"] = df["InterestMake"].apply(clean_interest_make)
    
    # Convert String features into type Category and float for the scikit Model to work with
    interest_make_categories = df["InterestMake"].astype("category").cat.categories
    interest_make_mapping = {cat: idx for idx, cat in enumerate(interest_make_categories)}
    df["InterestMake"] = df["InterestMake"].map(interest_make_mapping)

    # Method to only get the first word in the InterestModel feature
    def clean_interest_model(interest_model):
      interest_model=str(interest_model).upper().split(" ")
        
      return interest_model[0]
    
    # Clean the InterestModel feature
    df["InterestModel"] = df["InterestModel"].apply(clean_interest_model)
    
    # Convert String features into type Category and float for the scikit Model to work with
    interest_model_categories = df["InterestModel"].astype("category").cat.categories
    interest_model_mapping = {cat: idx for idx, cat in enumerate(interest_model_categories)}
    df["InterestModel"] = df["InterestModel"].map(interest_model_mapping)

    # Method to check if the name is a full name
    def is_full_name(full_name):
      full_name = str(full_name).split(" ")

      if len(full_name) < 2:
        return 0
      else:
        return 1

    # Check if the name given is a valid full name
    df["IsFullName"] = df["OBSFullName"].apply(is_full_name)

    # Drop OBSFUllName
    df = df.drop("OBSFullName", axis=1)

    # Drop OBSEmail
    df = df.drop("OBSEmail", axis=1)

    # retrieves the company of the email domain
    def retrieve_email_company(email_domain):
        if pd.isna(email_domain):
            return "UNKNOWN"
        domain_array = email_domain.split(".")
        return domain_array[0]

    # retrieves the TLD of the email domain
    def retrieve_email_tld(email_domain):
        if pd.isna(email_domain):
            return "UNKNOWN"
        domain_array = email_domain.split(".")
        return ".".join(domain_array[1:])

    # Extract the comapny of controlling the email domain
    df["DomainCompany"] = df["Domain"].apply(retrieve_email_company)

    # Convert the string feature to a usable fload value
    domain_company_categories = df["DomainCompany"].astype("category").cat.categories
    domain_company_mapping = {cat: idx for idx, cat in enumerate(domain_company_categories)}
    df["DomainCompany"] = df["DomainCompany"].map(domain_company_mapping)

    # Extract the TLD from the Domain
    df["DomainTLD"] = df["Domain"].apply(retrieve_email_tld)

    # Convert string feature into usable float value
    domain_tld_categories = df["DomainTLD"].astype("category").cat.categories
    domain_tld_mapping = {cat: idx for idx, cat in enumerate(domain_tld_categories)}
    df["DomainTLD"] = df["DomainTLD"].map(domain_tld_mapping)

    # Drop domain feature
    df = df.drop("Domain", axis=1)

    # check if a cellphone prefix is valid
    def cell_prefix_is_valid(cell_prefix):
        # check if does not contain characters
        if not cell_prefix.isdigit():
            return 0
        else:
            # check if it is 2 digits
            if len(cell_prefix) != 2:
                return 0
            else:
                return 1

    # Check if the cellprefix is valid
    df["ValidPrefix"] = df["CellPrefix"].apply(cell_prefix_is_valid)

    # Convert string feature into usable float value
    cell_prefix_categories = df["CellPrefix"].astype("category").cat.categories
    cell_prefix_mapping = {cat: idx for idx, cat in enumerate(cell_prefix_categories)}
    df["CellPrefix"] = df["CellPrefix"].map(cell_prefix_mapping)

    # Drop the finance features
    df = df.drop(
    ["InFinanceProcessSystemApp", "FinanceApplied", "FinanceApproved"],
    axis=1
)
    
    df = df.drop("DealerCity", axis=1)
    df = df.drop("DomainTLD", axis=1)
    df = df.drop("IsFullName", axis=1)

    # Return the cleaned dataframe and the new mappings
    return df, {
    "dealer_mapping": dealer_mapping,
    "dealer_city_mapping": dealer_city_mapping,
    "lead_type_mapping": lead_type_mapping,
    "lead_source_mapping": lead_source_mapping,
    "seek_mapping": seek_mapping,
    "interest_make_mapping": interest_make_mapping,
    "interest_model_mapping": interest_model_mapping,
    "domain_company_mapping": domain_company_mapping,
    "domain_tld_mapping": domain_tld_mapping,
    "cell_prefix_mapping": cell_prefix_mapping
}



In [344]:
# Define function to clean and engineer the data
def clean_test_data(df):
    # Drop CustomerID
    df = df.drop("CustomerID", axis=1)

    #Drop LeadID
    df = df.drop("LeadID", axis=1)

    # Converting DTLeadCreated to type DateTime
    df["DTLeadCreated"] = pd.to_datetime(df["DTLeadCreated"])
    df["DayOfWeek"] = df["DTLeadCreated"].dt.dayofweek # Extracting The Day of the Week from the date
    
    # Converting DRLeadAllocated to type DateTime
    df["DTLeadAllocated"] = pd.to_datetime(df["DTLeadAllocated"])
    
    # Calculating and storing the difference between lead creation and lead allocation
    df["AllocationTime"] = (df["DTLeadAllocated"] - df["DTLeadCreated"]).dt.total_seconds()/60
    
    # Dropping date Lead Created and date lead allocated
    df = df.drop("DTLeadCreated", axis=1)
    df = df.drop("DTLeadAllocated", axis=1)

    # Method to extract the deaer locations from the dealer name
    def get_dealer_location(dealer_string):

        #remove the suffix and change to lowercase
        dealer_string = dealer_string.lower().split()

        if "multifranchise" in dealer_string:
            dealer_name = dealer_string[0]

            # check the common two word cities at the end
            if dealer_name == "cape" or dealer_name == "east" or dealer_name == "west":
                dealer_name = dealer_name + " " + dealer_string[1]

        elif "motus" in dealer_string:
            dealer_name = dealer_string[-1]

            # check the common two word cities at the end
            if dealer_name == "town" or dealer_name == "rand":
                dealer_name = dealer_string[-2] + " " + dealer_name

        else:
            dealer_name = dealer_string[-1]

            # check the common two word cities at the end
            if dealer_name == "town" or dealer_name == "rand":
                dealer_name = dealer_string[-2] + " " + dealer_name

        # check if a number was found
        if dealer_name.isdigit():
            return dealer_string[-3]
        else:
            return dealer_name


    # Extract the Dealer's city
    df["DealerCity"] =df["Dealer"].apply(get_dealer_location)

    # Convert String features into type Category and float for the scikit Model to work with
    df["Dealer"] = df["Dealer"].map(dealer_mapping).fillna(-1)
    df["DealerCity"] = df["DealerCity"].map(mappings["dealer_city_mapping"]).fillna(-1)

    # Convert String features into type Category and float for the scikit Model to work with
    df["LeadType"] = df["LeadType"].map(mappings["lead_type_mapping"]).fillna(-1)
    #test_data["LeadSource"] = test_data["LeadSource"].astype("category").cat.codes.astype("float")
    df["LeadSource"] = df["LeadSource"].map(mappings["lead_source_mapping"]).fillna(-1)

    # Convert String features into type Category and float for the scikit Model to work with
    df["Seek"] = df["Seek"].map(mappings["seek_mapping"]).fillna(-1)

    # Method to get cleaned InterestMake
    def clean_interest_make(interest_make):
      interest_make = str(interest_make).upper().split(" ")
      interest_make = interest_make[0].split("-")
      return interest_make[0]

    # Clean InterestMake feature
    df["InterestMake"] = df["InterestMake"].apply(clean_interest_make)
    
    # Convert String features into type Category and float for the scikit Model to work with
    df["InterestMake"] = df["InterestMake"].map(mappings["interest_make_mapping"]).fillna(-1)

    # Method to only get the first word in the InterestModel feature
    def clean_interest_model(interest_model):
      interest_model=str(interest_model).upper().split(" ")
        
      return interest_model[0]
    
    # Clean the InterestModel feature
    df["InterestModel"] = df["InterestModel"].apply(clean_interest_model)
    
    # Convert String features into type Category and float for the scikit Model to work with
    df["InterestModel"] = df["InterestModel"].map(mappings["interest_model_mapping"]).fillna(-1)

    # Method to check if the name is a full name
    def is_full_name(full_name):
      full_name = str(full_name).split(" ")

      if len(full_name) < 2:
        return 0
      else:
        return 1

    # Check if the name given is a valid full name
    df["IsFullName"] = df["OBSFullName"].apply(is_full_name)

    # Drop OBSFUllName
    df = df.drop("OBSFullName", axis=1)

    # Drop OBSEmail
    df = df.drop("OBSEmail", axis=1)

    # retrieves the company of the email domain
    def retrieve_email_company(email_domain):
        if pd.isna(email_domain):
            return "UNKNOWN"
        domain_array = email_domain.split(".")
        return domain_array[0]

    # retrieves the TLD of the email domain
    def retrieve_email_tld(email_domain):
        if pd.isna(email_domain):
            return "UNKNOWN"
        domain_array = email_domain.split(".")
        return ".".join(domain_array[1:])

    # Extract the comapny of controlling the email domain
    df["DomainCompany"] = df["Domain"].apply(retrieve_email_company)

    # Convert the string feature to a usable fload value
    df["DomainCompany"] = df["DomainCompany"].map(mappings["domain_company_mapping"]).fillna(-1)

    # Extract the TLD from the Domain
    df["DomainTLD"] = df["Domain"].apply(retrieve_email_tld)

    # Convert string feature into usable float value
    df["DomainTLD"] = df["DomainTLD"].map(mappings["domain_tld_mapping"]).fillna(-1)

    # Drop domain feature
    df = df.drop("Domain", axis=1)

    # check if a cellphone prefix is valid
    def cell_prefix_is_valid(cell_prefix):
        # check if does not contain characters
        if not cell_prefix.isdigit():
            return 0
        else:
            # check if it is 2 digits
            if len(cell_prefix) != 2:
                return 0
            else:
                return 1

    # Check if the cellprefix is valid
    df["ValidPrefix"] = df["CellPrefix"].apply(cell_prefix_is_valid)

    # Convert string feature into usable float value
    df["CellPrefix"] = df["CellPrefix"].map(mappings["cell_prefix_mapping"]).fillna(-1)
    df = df.drop("DealerCity", axis=1)
    df = df.drop("DomainTLD", axis=1)
    df = df.drop("IsFullName", axis=1)

    # Return the cleaned dataframe
    return df



In [345]:
# Clean Train data
dfTrain, mappings = clean_train_data(dfTrain)

In [346]:
# Set target as VehiclesSold and remove from main dataframe
target = dfTrain["VehicleSold"]
dfTrain = dfTrain.drop("VehicleSold", axis=1)

In [347]:
# Store LeadID of Testing data
test_lead_id = dfTest["LeadID"]

# Clean Test data
dfTest = clean_test_data(dfTest)

In [348]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (average_precision_score,log_loss,precision_recall_curve,confusion_matrix)

In [349]:
#Split the data for training
X_train,X_test,y_train,y_test = train_test_split(
    dfTrain,
    target,
    test_size=0.2,
    stratify=target,
    random_state=2
)

In [350]:
#Create and train a random forest
rnd_clf= RandomForestClassifier(n_estimators=500 ,max_leaf_nodes=32 ,n_jobs=-1 ,class_weight='balanced',random_state=42)

rnd_clf.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', max_leaf_nodes=32,
                       n_estimators=500, n_jobs=-1, random_state=42)

In [351]:
import numpy as np

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    average_precision_score,
    log_loss,
    roc_auc_score,
    precision_recall_curve,
    confusion_matrix
)

In [352]:
# Convert potential categorical columns to string
potential_categories = [
    "Dealer",
    "LeadSource",
    "LeadType",
    "Seek",
    "InterestMake",
    "InterestModel",
    "CellPrefix",
    #"DealerCity",
    "DomainCompany",
    #"DomainTLD",

]


for col in potential_categories:
    if col in dfTrain.columns:
        dfTrain[col] = dfTrain[col].astype(str)

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    dfTrain,
    target,
    test_size=0.2,
    stratify=target,
    random_state=42
)


print("Training shape:", X_train.shape)
print("Validation shape:", X_val.shape)

Training shape: (58628, 14)
Validation shape: (14658, 14)


In [353]:
# Identify categorical features
categorical_features = [col for col in potential_categories if col in X_train.columns]
print("Categorical features for CatBoost:", categorical_features)

# Create Pools for CatBoost
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)


# Calculate class weight for imbalance
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
print("Scale_pos_weight: " ,scale_pos_weight)


#Train model with my best parameters

# Best parameters from  tuning
best_params = {
    'iterations': 300,
    'learning_rate': 0.05,
    'depth': 5,
    'l2_leaf_reg': 1
}

# Instantiate final model
catboost_model = CatBoostClassifier(
    iterations=best_params['iterations'],
    learning_rate=best_params['learning_rate'],
    depth=best_params['depth'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    eval_metric='PRAUC',
    scale_pos_weight=scale_pos_weight,
    cat_features=categorical_features,
    random_state=42,
    verbose=50,
    early_stopping_rounds=20
)


# Train model
catboost_model.fit(train_pool, eval_set=val_pool)

Categorical features for CatBoost: ['Dealer', 'LeadSource', 'LeadType', 'Seek', 'InterestMake', 'InterestModel', 'CellPrefix', 'DomainCompany']
Scale_pos_weight:  21.706429124709526
0:	learn: 0.6065351	test: 0.6345360	best: 0.6345360 (0)	total: 124ms	remaining: 37.2s
50:	learn: 0.6558795	test: 0.6537792	best: 0.6540364 (48)	total: 5.33s	remaining: 26s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6544809913
bestIteration = 79

Shrink model to first 80 iterations.


In [354]:
feat_imp = catboost_model.get_feature_importance(prettified=True)
print(feat_imp)

           Feature Id  Importances
0       InterestModel    19.034998
1            LeadType    15.512226
2        InterestMake    14.851429
3          CellPrefix    10.820921
4          LeadSource     9.337074
5                Seek     8.505684
6       DomainCompany     6.866206
7              Dealer     6.066332
8           DayOfWeek     3.957596
9       HourOfEnquiry     1.695584
10     AllocationTime     1.671012
11       DayOfEnquiry     0.803297
12  CellPhoneNoLength     0.611740
13        ValidPrefix     0.265901


In [355]:
feature_importance = pd.DataFrame({
    "Feature Id": catboost_model.feature_names_,
    "Importances": catboost_model.get_feature_importance()
})
print(feature_importance.sort_values("Importances", ascending=False))

           Feature Id  Importances
5       InterestModel    19.034998
2            LeadType    15.512226
4        InterestMake    14.851429
6          CellPrefix    10.820921
1          LeadSource     9.337074
3                Seek     8.505684
12      DomainCompany     6.866206
0              Dealer     6.066332
10          DayOfWeek     3.957596
8       HourOfEnquiry     1.695584
11     AllocationTime     1.671012
9        DayOfEnquiry     0.803297
7   CellPhoneNoLength     0.611740
13        ValidPrefix     0.265901


In [356]:
# Get  columns used for the original RF
rf_features = rnd_clf.feature_names_in_.tolist()

# Prepare the validation set with same columns
X_val_for_rf = X_val[rf_features]

# Predict with Random Forest
y_pred_probs_rf = rnd_clf.predict_proba(X_val_for_rf)[:, 1]

# CatBoost predictions
y_pred_probs_catboost = catboost_model.predict_proba(X_val)[:, 1]

# Blended probabilities (50-50)
mixed_probs = 0.2 * y_pred_probs_catboost + 0.8 * y_pred_probs_rf

# Evaluate blended model
pr_auc_mixed= average_precision_score(y_val,mixed_probs)
print("Blended PR-AUC: ",pr_auc_mixed)


Blended PR-AUC:  0.09863403261597034


In [357]:
#Create copy of cleaned Test Data
dfTest_copy = dfTest.copy()

In [358]:
# Ensure test data has the same columns as dfTrain
dfTest_rf = dfTest[rf_features]

# Convert CatBoost categorical features to string
for col in categorical_features:
    if col in dfTest.columns:
        dfTest[col] = dfTest[col].astype(str)



In [359]:
# Predict probabilities using Random Forest
test_probs_rf = rnd_clf.predict_proba(dfTest_rf)[:, 1]

# Predict probabilities using CatBoost
test_probs_catboost = catboost_model.predict_proba(dfTest)[:, 1]

In [360]:
# Blend the probabilities
probabilities = 0.2 * test_probs_catboost + 0.8 * test_probs_rf

In [361]:
# Define submission function
def save_submission(df: pd.DataFrame, probs: pd.Series | np.ndarray):
    submission = pd.DataFrame({
        "LeadID": test_lead_id.values,
        "VehicleSoldProbability": probs
    })
    submission.to_csv("submission.csv", index=False)

In [362]:
# 6. Save your final predictions
save_submission(dfTest, probabilities)